In [1]:
import pandas as pd
from dotenv import load_dotenv
import os
import requests
import time
import hashlib
import hmac
import plotly.express as px
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import seaborn as sns


os.environ.pop("API_KEY")
os.environ.pop("SECRET_KEY")

load_dotenv()

API_KEY = os.environ["API_KEY"]
API_SECRET = os.environ["SECRET_KEY"]

# Helper Functions

In [2]:
def boilerplate(params, endpoint):
    BASE_URL = 'https://api.binance.com'

    # Create the signature using HMAC-SHA256
    query_string = '&'.join([f"{key}={value}" for key, value in params.items()])
    signature = hmac.new(
        API_SECRET.encode('utf-8'),
        query_string.encode('utf-8'),
        hashlib.sha256
    ).hexdigest()

    # Add the signature to the parameters
    params['signature'] = signature

    # Define the headers with API key
    headers = {
        'X-MBX-APIKEY': API_KEY
    }

    # Send the GET request
    response = requests.get(BASE_URL + endpoint, headers=headers, params=params)

    # Check response status and print the result
    if response.status_code == 200:
        # print(response.json())
        return response.json()
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return Exception(f"Error: {response.status_code}, {response.text}")
    
def boilerplate1(params, endpoint):
    BASE_URL = 'https://api.binance.com'

    # Create the signature using HMAC-SHA256
    query_string = '&'.join([f"{key}={value}" for key, value in params.items()])
    # signature = hmac.new(
    #     API_SECRET.encode('utf-8'),
    #     query_string.encode('utf-8'),
    #     hashlib.sha256
    # ).hexdigest()

    # # Add the signature to the parameters
    # params['signature'] = signature

    # Define the headers with API key
    headers = {
        'X-MBX-APIKEY': API_KEY
    }

    # Send the GET request
    response = requests.get(BASE_URL + endpoint, headers=headers, params=params)

    # Check response status and print the result
    if response.status_code == 200:
        # print(response.json())
        return response.json()
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return Exception(f"Error: {response.status_code}, {response.text}")
    

def boilerplate_post(params, endpoint):
    BASE_URL = 'https://api.binance.com'

    # Create the signature using HMAC-SHA256
    query_string = '&'.join([f"{key}={value}" for key, value in params.items()])
    signature = hmac.new(
        API_SECRET.encode('utf-8'),
        query_string.encode('utf-8'),
        hashlib.sha256
    ).hexdigest()

    # Add the signature to the parameters
    params['signature'] = signature

    # Define the headers with API key
    headers = {
        'X-MBX-APIKEY': API_KEY
    }

    # Send the POST request
    response = requests.post(BASE_URL + endpoint, headers=headers, params=params)

    # Check response status and print the result
    if response.status_code == 200:
        # print(response.json())
        return response.json()
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return Exception(f"Error: {response.status_code}, {response.text}")

# Dual Investment Strategy
- **Put** Indicates that you are trying to buy the crypto product/ coin/ asset with stable coin. It means you are trying to buy Low
- **Call** Indicates that you are trying to Sell the crypto product/ coin/ asset for stable coin. It means you are trying to Sell High


In [3]:
def get_DCI_products(optionType, exercisedCoin, investCoin, pageSize, pageIndex):

    # Define the endpoint and base URL
    endpoint = '/sapi/v1/dci/product/list'

    # Define request parameters
    params = {
        'optionType': optionType,  #'CALL' or 'PUT'
        'exercisedCoin': exercisedCoin,  # Target exercised asset
        'investCoin': investCoin,  # Asset used for subscribing
        'pageSize': pageSize,  # Optional
        'pageIndex': pageIndex,  # Optional
        'recvWindow': 60000,  # Optional
        'timestamp': int(time.time() * 1000)  # Current timestamp in milliseconds
    }

    return boilerplate(params, endpoint)


In [4]:
def get_DCI_account():

    # Define the endpoint and base URL
    endpoint = '/sapi/v1/dci/product/accounts'

    # Define request parameters
    params = {
        'recvWindow': 60000,  # Optional
        'timestamp': int(time.time() * 1000)  # Current timestamp in milliseconds
    }

    return boilerplate(params, endpoint)



In [5]:
def get_DCI_positions():

    # Define the endpoint and base URL
    endpoint = '/sapi/v1/dci/product/positions'

    # Define request parameters
    params = {
        'recvWindow': 60000,  # Optional
        'timestamp': int(time.time() * 1000)  # Current timestamp in milliseconds
    }
    return boilerplate(params, endpoint)


In [6]:
def get_price_historical(symbol, interval):
    
    # Define the endpoint and base URL
    endpoint = '/api/v3/klines'

    # Define request parameters
    params = {
        'symbol': symbol,   
        'interval': interval  
    }
    return boilerplate1(params, endpoint)

In [7]:
def get_full_Assets():

    # Define the endpoint and base URL
    endpoint = '/sapi/v3/asset/getUserAsset'

    # Define request parameters
    params = {
        'timestamp': int(time.time() * 1000)  # Current timestamp in milliseconds
    }

    return boilerplate_post(params, endpoint)

In [8]:
def get_DualInvestments_summary():

    # Define the endpoint and base URL
    endpoint = '/sapi/v1/dci/product/accounts'

    # Define request parameters
    params = {
        'timestamp': int(time.time() * 1000)  # Current timestamp in milliseconds
    }

    return boilerplate(params, endpoint)

In [9]:
def get_price(symbol):
    # Define the endpoint and base URL
    endpoint = '/api/v3/avgPrice'

    # Define request parameters
    params = {
        'symbol': symbol
    }

    return boilerplate1(params, endpoint)

def get_all_prices():
    symbols = ["BTCUSDC", "ETHUSDC", "SOLUSDC", "BNBUSDC", "DOGEUSDC"]
    symbol_price_dict = {}
    for symbol in symbols:
        price = float(get_price(symbol)["price"])
        symbol_price_dict[symbol] = price
    return symbol_price_dict

In [10]:
def get_full_DualInvestments(settled):

    # Define the endpoint and base URL
    endpoint = '/sapi/v1/dci/product/positions'

    # Define request parameters
    params = {
        'timestamp': int(time.time() * 1000)  # Current timestamp in milliseconds
    }
    # Await boilerplate function if it's an async function
    response = boilerplate(params, endpoint)
    list_of_DI = response.get("list", [])
    
    # Use filter correctly to match items with settled status
    # value = list(filter(lambda x: x["purchaseStatus"], list_of_DI))
    value = list(filter(lambda x: x["purchaseStatus"] == settled, list_of_DI))
    return value

prices = get_all_prices()

    
def get_full_DualInvestments_agg(settled):
    prices = get_all_prices()
    value = get_full_DualInvestments(settled)
    final = 0
    for item in value:
            final += float(item["subscriptionAmount"])* prices.get(f"{item['investCoin']}USDC", 1)
    return final


            

    

In [11]:
get_full_DualInvestments_agg("PURCHASE_SUCCESS")

Error: 400, {"code":-2015,"msg":"Invalid API-key, IP, or permissions for action."}


AttributeError: 'Exception' object has no attribute 'get'

In [12]:
get_DualInvestments_summary()

Error: 400, {"code":-2015,"msg":"Invalid API-key, IP, or permissions for action."}


Exception('Error: 400, {"code":-2015,"msg":"Invalid API-key, IP, or permissions for action."}')

# Runner


In [13]:
target = "ETH"
currency = "USDC"
target_currency = f"{target}{currency}"
print(target_currency)

ETHUSDC


In [14]:
data = get_price_historical(target_currency, "1d")
columns = [
    "Kline Open Time", "Open Price", "High Price", "Low Price", 
    "Close Price", "Volume", "Kline Close Time", "Quote Asset Volume", 
    "Number of Trades", "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume"
]

cleaned_data = [row[:-1] for row in data]  # Remove the last element from each row
df_history = pd.DataFrame(cleaned_data, columns=columns)

df_history['Kline Open Time'] = pd.to_datetime(df_history['Kline Open Time'],unit='ms')
df_history['Kline Close Time'] = pd.to_datetime(df_history['Kline Close Time'],unit='ms')

df_history["Open Price"] = pd.to_numeric(df_history["Open Price"])
df_history["High Price"] = pd.to_numeric(df_history["High Price"])
df_history["Low Price"] = pd.to_numeric(df_history["Low Price"])
df_history["Close Price"] = pd.to_numeric(df_history["Close Price"])
df_history["Volume"] = pd.to_numeric(df_history["Volume"])
df_history["Quote Asset Volume"] = pd.to_numeric(df_history["Quote Asset Volume"])
df_history["Taker Buy Base Asset Volume"] = pd.to_numeric(df_history["Taker Buy Base Asset Volume"])
df_history["Taker Buy Quote Asset Volume"] = pd.to_numeric(df_history["Taker Buy Quote Asset Volume"])



df_copy = df_history

In [15]:
curr_price = df_history.tail(1)["Close Price"].iloc[0]
curr_price

np.float64(2944.3)

In [16]:
df_price_movements_column = []
for n in range(1, 31):
    df_price_movements_column.append(f'Price Movement {n} days')
    df_copy[f'Price Movement {n} days'] = (df_copy['Close Price'].shift(n) - df_copy['Close Price']) / df_copy['Close Price']


df_price_movements = df_copy.dropna()
df_price_movements=df_price_movements[df_price_movements_column]
df_price_movements


,Price Movement 1 days,Price Movement 2 days,Price Movement 3 days,Price Movement 4 days,Price Movement 5 days,Price Movement 6 days,Price Movement 7 days,Price Movement 8 days,Price Movement 9 days,Price Movement 10 days,...,Price Movement 21 days,Price Movement 22 days,Price Movement 23 days,Price Movement 24 days,Price Movement 25 days,Price Movement 26 days,Price Movement 27 days,Price Movement 28 days,Price Movement 29 days,Price Movement 30 days
30,-0.007401,-0.001558,-0.008628,-0.012827,0.007865,-0.004306,0.009295,0.008708,0.007721,0.012560,...,-0.001649,-0.014636,0.019331,0.033156,0.043438,0.033449,0.026876,0.031491,-0.011963,-0.024704
31,-0.003334,-0.010710,-0.004887,-0.011933,-0.016118,0.004504,-0.007626,0.005929,0.005344,0.004361,...,-0.007961,-0.004978,-0.017921,0.015932,0.029711,0.039959,0.030003,0.023452,0.028052,-0.015257
32,0.010202,0.006833,-0.000618,0.005265,-0.001853,-0.006081,0.014752,0.002498,0.016192,0.015601,...,0.000666,0.002160,0.005173,-0.007902,0.026297,0.040216,0.050568,0.040511,0.033893,0.038540
33,0.002855,0.013086,0.009708,0.002236,0.008135,0.000997,-0.003243,0.017649,0.005361,0.019093,...,0.013189,0.003523,0.005021,0.008043,-0.005070,0.029227,0.043186,0.053568,0.043482,0.036845
34,-0.008975,-0.006145,0.003994,0.000646,-0.006759,-0.000913,-0.007987,-0.012189,0.008516,-0.003663,...,0.002776,0.004095,-0.005483,-0.003999,-0.001004,-0.013999,0.019990,0.033823,0.044112,0.034117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.024813,0.040240,0.047257,0.049276,0.108997,0.100629,0.070539,0.045087,0.034404,0.016315,...,0.096883,0.029640,0.032889,0.017421,-0.005235,-0.011781,0.017742,0.010717,0.017888,0.007163
496,-0.010436,0.014118,0.029384,0.036328,0.038326,0.097424,0.089143,0.059367,0.034181,0.023609,...,0.076214,0.085436,0.018894,0.022110,0.006803,-0.015617,-0.022094,0.007121,0.000169,0.007266
497,-0.110938,-0.120216,-0.098386,-0.084814,-0.078641,-0.076864,-0.024322,-0.031685,-0.058158,-0.080549,...,-0.041737,-0.043180,-0.034980,-0.094140,-0.091281,-0.104890,-0.124822,-0.130581,-0.104607,-0.110788
498,-0.059677,-0.163995,-0.172719,-0.152192,-0.139430,-0.133625,-0.131954,-0.082548,-0.089471,-0.114364,...,-0.100984,-0.098924,-0.100280,-0.092570,-0.148199,-0.145511,-0.158307,-0.177051,-0.182465,-0.158042


In [17]:
summary_stats = df_price_movements.describe()
skewness = df_price_movements.skew()
kurtosis = df_price_movements.kurtosis()

In [18]:
# 1. Visualizing Price Movements Using Line Plot
fig_line = go.Figure()

# Create a line for each column
for col in df_price_movements.columns:
    fig_line.add_trace(go.Scatter(x=df_price_movements.index, y=df_price_movements[col], mode='lines', name=col))

# Update layout
fig_line.update_layout(
    title=f"Daily Price Movement Percentage {target}",
    xaxis_title="Time",
    yaxis_title="Percentage Movement",
    template='plotly'
)

# Show the line plot
fig_line.show()

# 2. Visualizing the Distribution with Box Plot
fig_box = px.box(df_price_movements, title=f"Distribution of Price Movements {target}")

# Show the box plot
fig_box.show()

# Test (required input)

In [20]:
target = "ETH"
currency = "USDC"
USDamt = 31431.87390215

## Sell High
# AssetItem = target
# TargetItem = currency
# Direction = "CALL"

## Buy Low
TargetItem= target
AssetItem = currency
Direction = "PUT"

# Dual Investment tracking

In [25]:
def get_DualInvestment_asset(Direction, TargetItem, AssetItem):
    print(Direction, TargetItem, AssetItem)
    first = get_DCI_products(Direction, TargetItem, AssetItem, pageIndex=1, pageSize=1)
    full_list = first["list"]
    total = first["total"]
    print(total)

    for i in range(total//100+1):
        try:
            curr = get_DCI_products(Direction, TargetItem, AssetItem, pageIndex=i+1, pageSize=100)
            full_list.extend(curr["list"])
        except Exception as e:
            print(f"ERROR found: {e}")

    print("---------------------------")
    print(len(full_list))
    df = pd.DataFrame(full_list)
    df["curr_price"]  = curr_price
    df["strikePrice"] = pd.to_numeric(df["strikePrice"])

    df["Percent_to_strikeprice"] = 100*(df["strikePrice"] - df["curr_price"])/df["curr_price"]

    df = df.sort_values(by=['apr', 'duration'], ascending=[False, True])
    df["strikePrice"] = df.strikePrice.astype(float)
    df["apr"] = df.apr.astype(float)
    df["1000return"] = df["apr"]*df["duration"]*1000/365
    df["USDamt"] = df["1000return"]*USDamt/1000
    return df

df = get_DualInvestment_asset(Direction, TargetItem, AssetItem)

PUT ETH USDC
339
---------------------------
340


In [26]:
data = []
for cryptoAsset in ["BTC", "ETH", "SOL"]:
    print(f"Extracting for {cryptoAsset}")
    dataframe = get_DualInvestment_asset("PUT", cryptoAsset ,"USDC")
    data.append(dataframe)
df = pd.concat(data, ignore_index=True, sort=False)

Extracting for BTC
PUT BTC USDC
484
---------------------------
485
Extracting for ETH
PUT ETH USDC
339
---------------------------
340
Extracting for SOL
PUT SOL USDC
189
---------------------------
190


In [27]:


df

,id,investCoin,exercisedCoin,strikePrice,duration,settleDate,purchaseDecimal,purchaseEndTime,canPurchase,apr,...,minAmount,maxAmount,createTimestamp,optionType,isAutoCompoundEnable,autoCompoundPlanList,curr_price,Percent_to_strikeprice,1000return,USDamt
0,1562183,USDC,BTC,90500.0,6,1732262400000,8,1732204800000,True,1.0331,...,0.1,4400000,1731508456000,PUT,True,"[STANDARD, ADVANCE]",2944.3,2973.735693,16.982466,533.790722
1,1562183,USDC,BTC,90500.0,6,1732262400000,8,1732204800000,True,1.0331,...,0.1,4400000,1731508456000,PUT,True,"[STANDARD, ADVANCE]",2944.3,2973.735693,16.982466,533.790722
2,1562190,USDC,BTC,90500.0,3,1732003200000,8,1731945600000,True,0.8954,...,0.1,7240000,1731508458000,PUT,True,"[STANDARD, ADVANCE]",2944.3,2973.735693,7.359452,231.321369
3,1562111,USDC,BTC,90000.0,6,1732262400000,8,1732204800000,True,0.8750,...,0.1,4400000,1731508334000,PUT,True,"[STANDARD, ADVANCE]",2944.3,2956.753728,14.383562,452.102296
4,1549983,USDC,BTC,89500.0,6,1732262400000,8,1732204800000,True,0.8050,...,0.1,4400000,1731366917000,PUT,True,"[STANDARD, ADVANCE]",2944.3,2939.771762,13.232877,415.934112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1010,1520124,USDC,SOL,195.0,104,1740729600000,8,1740672000000,True,0.0373,...,0.1,5000000,1731000789000,PUT,True,"[STANDARD, ADVANCE]",2944.3,-93.377034,10.627945,334.056234
1011,1444404,USDC,SOL,167.5,6,1732262400000,8,1732204800000,True,0.0372,...,0.1,418750,1729466894000,PUT,True,"[STANDARD, ADVANCE]",2944.3,-94.311042,0.611507,19.220806
1012,1443416,USDC,SOL,165.0,76,1738310400000,8,1738252800000,True,0.0367,...,0.1,412500,1729462090000,PUT,True,"[STANDARD, ADVANCE]",2944.3,-94.395951,7.641644,240.191185
1013,1279675,USDC,SOL,160.0,76,1738310400000,8,1738252800000,True,0.0366,...,0.1,400000,1723128550000,PUT,True,"[STANDARD, ADVANCE]",2944.3,-94.565771,7.620822,239.536714


In [28]:
df_copy = df

# df_copy = df[df["strikePrice"]>=2200]
# df_copy = df_copy[df_copy["duration"]<=25]
# df_copy = df_copy[df_copy["apr"]>=0.2]

BTC_Buy_limit = 60000
ETH_Buy_limit = 2400
SOL_Buy_limit = 140

# BTC
# df_copy = df_copy[(df_copy["strikePrice"]<=BTC_Buy_limit) & (df_copy["exercisedCoin"]=="BTC")]
# df_copy = df_copy[df_copy["strikePrice"]>=55000 ]

# ETH
# df_copy = df_copy[(df_copy["strikePrice"]<=ETH_Buy_limit) & (df_copy["exercisedCoin"]=="ETH")]
# df_copy = df_copy[df_copy["strikePrice"]>=2150 ]


# SOL
# df_copy = df_copy[(df_copy["strikePrice"]<=SOL_Buy_limit) & (df_copy["exercisedCoin"]=="SOL")]
# df_copy = df_copy[df_copy["strikePrice"]>=2150 ]

#resultant
# df_copy = df_copy[(df_copy["strikePrice"]<=SOL_Buy_limit) & (df_copy["exercisedCoin"]=="SOL") or (df_copy["strikePrice"]<=ETH_Buy_limit) & (df_copy["exercisedCoin"]=="ETH") or (df_copy["strikePrice"]<=BTC_Buy_limit) & (df_copy["exercisedCoin"]=="BTC")]




fig = px.scatter(df_copy, x="duration", y="apr", color='exercisedCoin', hover_data=["1000return", "USDamt", "Percent_to_strikeprice"])
fig.show()

In [30]:
df_copy = df

# df_copy = df[df["strikePrice"]>=2200]
# df_copy = df_copy[df_copy["duration"]<=25]
# df_copy = df_copy[df_copy["apr"]>=0.2]

# df_copy = df_copy[df_copy["strikePrice"]<=60000 ]
# df_copy = df_copy[df_copy["strikePrice"]>=55000 ]

# df_copy = df_copy[df_copy["strikePrice"]<=2500 ]
# df_copy = df_copy[df_copy["strikePrice"]>=2150 ]

# df_copy = df_copy[df_copy["duration"]== 5]

# df_copy = df_copy[df_copy["exercisedCoin"]== "ETH"]


fig = px.line(df_copy, x="strikePrice", y="apr", color="exercisedCoin", hover_data=["1000return", "USDamt", "Percent_to_strikeprice"], markers=True)
fig.show()

# Full Assets

In [101]:
get_full_Assets()

[{'asset': 'BTC',
  'free': '0.0000088',
  'locked': '0',
  'freeze': '0',
  'withdrawing': '0',
  'ipoable': '0',
  'btcValuation': '0'},
 {'asset': 'DOGE',
  'free': '0.78410331',
  'locked': '0',
  'freeze': '0',
  'withdrawing': '0',
  'ipoable': '0',
  'btcValuation': '0'},
 {'asset': 'ETH',
  'free': '0.00007395',
  'locked': '0',
  'freeze': '0',
  'withdrawing': '0',
  'ipoable': '0',
  'btcValuation': '0'},
 {'asset': 'FDUSD',
  'free': '2.75496977',
  'locked': '0',
  'freeze': '0',
  'withdrawing': '0',
  'ipoable': '0',
  'btcValuation': '0'},
 {'asset': 'SCR',
  'free': '2.49459523',
  'locked': '0',
  'freeze': '0',
  'withdrawing': '0',
  'ipoable': '0',
  'btcValuation': '0'},
 {'asset': 'SOL',
  'free': '0.0004542',
  'locked': '0',
  'freeze': '0',
  'withdrawing': '0',
  'ipoable': '0',
  'btcValuation': '0'},
 {'asset': 'USDC',
  'free': '0.0848962',
  'locked': '0',
  'freeze': '0',
  'withdrawing': '0',
  'ipoable': '0',
  'btcValuation': '0'},
 {'asset': 'USDT',


# Dual investment summary

In [128]:
get_full_DualInvestments("PURCHASE_SUCCESS")

<coroutine object get_full_DualInvestments at 0x1197ec640>

In [ ]:
[{'id': '12265969', 'investCoin': 'USDT', 'exercisedCoin': 'ETH', 'subscriptionAmount': '11465', 'strikePrice': '2275', 'duration': 6, 'settleDate': 1731052800000, 'purchaseStatus': 'PURCHASE_SUCCESS', 'apr': '0.3406', 'orderId': 20378124691, 'purchaseEndTime': 1730966400000, 'optionType': 'PUT'}, {'id': '12213666', 'investCoin': 'USDC', 'exercisedCoin': 'ETH', 'subscriptionAmount': '2261.8', 'strikePrice': '2275', 'duration': 12, 'settleDate': 1731052800000, 'purchaseStatus': 'PURCHASE_SUCCESS', 'apr': '0.3264', 'orderId': 20117788247, 'purchaseEndTime': 1730966400000, 'optionType': 'PUT'}, {'id': '12207859', 'investCoin': 'USDC', 'exercisedCoin': 'ETH', 'subscriptionAmount': '8499.8', 'strikePrice': '2275', 'duration': 13, 'settleDate': 1731052800000, 'purchaseStatus': 'PURCHASE_SUCCESS', 'apr': '0.4811', 'orderId': 20048577817, 'purchaseEndTime': 1730966400000, 'optionType': 'PUT'}, {'id': '12182253', 'investCoin': 'FDUSD', 'exercisedCoin': 'ETH', 'subscriptionAmount': '4402.9', 'strikePrice': '2300', 'duration': 15, 'settleDate': 1731052800000, 'purchaseStatus': 'PURCHASE_SUCCESS', 'apr': '0.243', 'orderId': 19988007170, 'purchaseEndTime': 1730966400000, 'optionType': 'PUT'}, {'id': '12182225', 'investCoin': 'ETH', 'exercisedCoin': 'USDC', 'subscriptionAmount': '7.1878', 'strikePrice': '2825', 'duration': 15, 'settleDate': 1731052800000, 'purchaseStatus': 'PURCHASE_SUCCESS', 'apr': '0.2344', 'orderId': 19987418079, 'purchaseEndTime': 1730966400000, 'optionType': 'CALL'}, {'id': '12149842', 'investCoin': 'FDUSD', 'exercisedCoin': 'ETH', 'subscriptionAmount': '4398.3', 'strikePrice': '2450', 'duration': 4, 'settleDate': 1729584000000, 'purchaseStatus': 'SETTLED', 'apr': '0.068', 'orderId': 19730202274, 'purchaseEndTime': 1729497600000, 'optionType': 'PUT', 'settlePrice': '2653.57733333', 'isExercised': False, 'settleAsset': 'FDUSD', 'settleAmount': '4401.37881'}, {'id': '12149546', 'investCoin': 'BTC', 'exercisedCoin': 'USDC', 'subscriptionAmount': '0.07455', 'strikePrice': '73000', 'duration': 42, 'settleDate': 1732867200000, 'purchaseStatus': 'PURCHASE_SUCCESS', 'apr': '0.3084', 'orderId': 19727613085, 'purchaseEndTime': 1732780800000, 'optionType': 'CALL'}, {'id': '12128715', 'investCoin': 'DOGE', 'exercisedCoin': 'USDT', 'subscriptionAmount': '78668', 'strikePrice': '0.145', 'duration': 14, 'settleDate': 1730448000000, 'purchaseStatus': 'SETTLED', 'apr': '0.1343', 'orderId': 19705871830, 'purchaseEndTime': 1730361600000, 'optionType': 'CALL', 'settlePrice': '0.15591433', 'isExercised': True, 'settleAsset': 'USDT', 'settleAmount': '11465.034986'}, {'id': '12098198', 'investCoin': 'BTC', 'exercisedCoin': 'USDC', 'subscriptionAmount': '0.07449', 'strikePrice': '69000', 'duration': 6, 'settleDate': 1729238400000, 'purchaseStatus': 'SETTLED', 'apr': '0.0495', 'orderId': 19457444166, 'purchaseEndTime': 1729152000000, 'optionType': 'CALL', 'settlePrice': '67964.22233333', 'isExercised': False, 'settleAsset': 'BTC', 'settleAmount': '0.07454959'}, {'id': '12098021', 'investCoin': 'USDC', 'exercisedCoin': 'ETH', 'subscriptionAmount': '8414.5', 'strikePrice': '2300', 'duration': 13, 'settleDate': 1729843200000, 'purchaseStatus': 'SETTLED', 'apr': '0.2848', 'orderId': 19454416573, 'purchaseEndTime': 1729756800000, 'optionType': 'PUT', 'settlePrice': '2475.01183333', 'isExercised': False, 'settleAsset': 'USDC', 'settleAmount': '8499.48645'}]
